In [1]:
import torch
import torchvision
import torchvision.transforms as transforms

import torch.nn as nn
import torch.nn.functional as F

Stem 영역을 세 구간으로 나누어 선언한다

In [2]:
class stem1(nn.Module):
    def __init__(self):
        super(stem1, self).__init__()
        self.a1 = nn.Sequential(
            nn.Conv2d(3, 32, 3, stride = 2),
            nn.ReLU(True),
            nn.Conv2d(32, 32, 3, stride = 1),
            nn.ReLU(True),
            nn.Conv2d(32, 64, 3, stride = 1, padding = 1),
            nn.ReLU(True),
        )
        self.a2 = nn.Sequential(
            nn.Conv2d(64, 96, 3, stride = 2),
            nn.ReLU(True),
        )
        self.a3 = nn.Sequential(
            nn.MaxPool2d(3, stride = 2),
        )
    def forward(self,x):
        y1 = self.a1(x)
        out1 = self.a2(y1)
        out2 = self.a3(y1)
        return torch.cat([out1,out2],1)       
        

In [31]:
class stem2(nn.Module):
    def __init__(self):
        super(stem2, self).__init__()
        self.a1 = nn.Sequential(
            nn.Conv2d(160, 64, 1, stride = 1),
            nn.ReLU(True),
            nn.Conv2d(64, 64, (7, 1), stride = 1,padding=(3,0)),
            nn.ReLU(True),
            nn.Conv2d(64, 64, (1, 7), stride = 1,padding=(0,3)),
            nn.ReLU(True),
            nn.Conv2d(64, 96, 3, stride = 1),
            nn.ReLU(True)
        )
        self.a2 = nn.Sequential(
            nn.Conv2d(160, 64, 1, stride = 1),
            nn.ReLU(True),
            nn.Conv2d(64, 96, 3, stride = 1),
            nn.ReLU(True),
        )
    def forward(self, x):
        y1 = self.a1(x)
        y2 = self.a2(x)
        return torch.cat([y1,y2],1)

In [4]:
class stem3(nn.Module):
    def __init__(self):
        super(stem3, self).__init__()
        self.a1 = nn.Sequential(
            nn.Conv2d(192, 192, 3, stride = 2),
            nn.ReLU(True),
        )
        self.a2 = nn.Sequential(
            nn.MaxPool2d(3, stride = 2)
        )
    def forward(self,x):
        y1 = self.a1(x)
        y2 = self.a2(x)
        return torch.cat([y1,y2],1)

4 x Inception-A 영역을 선언한다

In [35]:
class InceptionA(nn.Module):
    def __init__(self):
        super(InceptionA, self).__init__()
        self.a1 = nn.Sequential(
            nn.Conv2d(384, 64, 1, stride = 1),
            nn.ReLU(True),
            nn.Conv2d(64, 96, 3, stride = 1, padding=1),
            nn.ReLU(True),
            nn.Conv2d(96, 96, 3, stride = 1, padding=1),
            nn.ReLU(True),
        )
        self.a2 = nn.Sequential(
            nn.Conv2d(384, 64, 1, stride = 1),
            nn.ReLU(True),
            nn.Conv2d(64, 96, 3, stride = 1, padding=1),
            nn.ReLU(True),
        )
        self.a3 = nn.Sequential(
            nn.Conv2d(384, 96, 1, stride = 1),
            nn.ReLU(True),
        )
        self.a4 = nn.Sequential(
            nn.AvgPool2d(3, stride = 1, padding=1),
            nn.Conv2d(384, 96, 1, stride = 1),
            nn.ReLU(True),
        )
    def forward(self,x):
        y1 = self.a1(x)
        y2 = self.a2(x)
        y3 = self.a3(x)
        y4 = self.a4(x)
        return torch.cat([y1,y2,y3,y4],1)

ReductionA를 선언한다.

In [38]:
class ReductionA(nn.Module):
    def __init__(self):
        super(ReductionA, self).__init__()
        self.a1 = nn.Sequential(
            nn.Conv2d(384, 192, 1, stride = 1),
            nn.ReLU(True),
            nn.Conv2d(192, 224, 3, stride = 1, padding=1),
            nn.ReLU(True),
            nn.Conv2d(224, 256, 3, stride = 2),
            nn.ReLU(True),
        )
        self.a2 = nn.Sequential(
            nn.Conv2d(384, 384, 3, stride = 2),
            nn.ReLU(True),
        )
        self.a3 = nn.Sequential(
            nn.MaxPool2d(3, stride = 2),
        )
    
    def forward(self, x):
        y1 = self.a1(x)
        y2 = self.a2(x)
        y3 = self.a3(x)
        return torch.cat([y1,y2,y3],1)
        

7 x Inception-B를 선언한다.

In [41]:
class InceptionB(nn.Module):
    def __init__(self):
        super(InceptionB, self).__init__()
        self.a1 = nn.Sequential(
            nn.Conv2d(1024, 192, 1, stride = 1),
            nn.ReLU(True),
            nn.Conv2d(192, 192, (1, 7), stride = 1, padding=(0,3)),
            nn.ReLU(True),
            nn.Conv2d(192, 224, (7, 1), stride = 1, padding=(3,0)),
            nn.ReLU(True),
            nn.Conv2d(224, 224, (1, 7), stride = 1, padding=(0,3)),
            nn.ReLU(True),
            nn.Conv2d(224, 256, (7, 1), stride = 1, padding=(3,0)),
            nn.ReLU(True),
        )
        self.a2 = nn.Sequential(
            nn.Conv2d(1024, 192, 1, stride = 1),
            nn.ReLU(True),
            nn.Conv2d(192, 224, (1, 7), stride = 1, padding=(0,3)),
            nn.ReLU(True),
            nn.Conv2d(224, 256, (1, 7), stride = 1, padding=(0,3)),
            nn.ReLU(True),
        )
        self.a3 = nn.Sequential(
            nn.Conv2d(1024, 384, 1, stride = 1),
            nn.ReLU(True),
        )
        self.a4 = nn.Sequential(
            nn.AvgPool2d(3, stride = 1, padding = 1),
            nn.Conv2d(1024, 128, 1, stride = 1),
            nn.ReLU(True),
        )
    
    def forward(self,x):
        y1 = self.a1(x)
        y2 = self.a2(x)
        y3 = self.a3(x)
        y4 = self.a4(x)
        return torch.cat([y1,y2,y3,y4],1)
        
    

ReductionB를 선언한다.

In [44]:
class ReductionB(nn.Module):
    def __init__(self):
        super(ReductionB, self).__init__()
        self.a1 = nn.Sequential(
            nn.Conv2d(1024, 256, 1, stride = 1),
            nn.ReLU(True),
            nn.Conv2d(256, 256, (1, 7), stride = 1, padding=(0,3)),
            nn.ReLU(True),
            nn.Conv2d(256, 320, (7, 1), stride = 1, padding=(3,0)),
            nn.ReLU(True),
            nn.Conv2d(320, 320, 3, stride = 2),
            nn.ReLU(True),
        )
        self.a2 = nn.Sequential(
            nn.Conv2d(1024, 192, 1, stride = 1),
            nn.ReLU(True),
            nn.Conv2d(192, 192, 3, stride = 2),
            nn.ReLU(True),
        )
        self.a3 = nn.Sequential(
            nn.MaxPool2d(3, stride = 2),
        )
    
    def forward(self, x):
        y1 = self.a1(x)
        y2 = self.a2(x)
        y3 = self.a3(x)
        return torch.cat([y1,y2,y3],1)


3 x Inception-C를 선언한다.

In [51]:
class InceptionC(nn.Module):
    def __init__(self):
        super(InceptionC, self).__init__()
        self.a1 = nn.Sequential(
            nn.Conv2d(1536, 384, 1, stride = 1),
            nn.ReLU(True),
            nn.Conv2d(384, 448, (1, 3), stride = 1, padding=(0,1)),
            nn.ReLU(True),
            nn.Conv2d(448, 512, (3, 1), stride = 1, padding=(1,0)),
            nn.ReLU(True),
        )
        self.a2 = nn.Sequential(
            nn.Conv2d(512, 256, (1, 3), stride = 1, padding=(0,1)),
            nn.ReLU(True),
        )
        self.a3 = nn.Sequential(
            nn.Conv2d(512, 256, (3, 1), stride = 1, padding=(1,0)),
            nn.ReLU(True),
        )
        self.b1 = nn.Sequential(
            nn.Conv2d(1536, 384, 1, stride = 1),
            nn.ReLU(True),
        )
        self.b2 = nn.Sequential(
            nn.Conv2d(384, 256, (3, 1), stride = 1, padding=(1,0)),
            nn.ReLU(True),
        )
        self.b3 = nn.Sequential(
            nn.Conv2d(384, 256, (1, 3), stride = 1, padding=(0,1)),
            nn.ReLU(True),
        )
        self.c1 = nn.Sequential(
            nn.Conv2d(1536, 256, 1, stride = 1),
            nn.ReLU(True),
        )
        self.d1 = nn.Sequential(
            nn.AvgPool2d(3,stride = 1, padding = 1),
            nn.Conv2d(1536, 256, 1, stride = 1),
            nn.ReLU(True),
        )
        
        
        
    def forward(self, x):
        m = self.a1(x)
        y1 = self.a2(m)
        y2 = self.a3(m)
        n = self.b1(x)
        y3 = self.b2(n)
        y4 = self.b3(n)
        y5 = self.c1(x)
        y6 = self.d1(x)
        return torch.cat([y1,y2,y3,y4,y5,y6],1)      


In [52]:
class Inceptionv4(nn.Module):
    def __init__(self):
        super(Inceptionv4,self).__init__()
        
        self.a1 = stem1()
        self.a2 = stem2()
        self.a3 = stem3()
        self.a4 = InceptionA()
        self.a5 = ReductionA()
        self.a6 = InceptionB()
        self.a7 = ReductionB()
        self.a8 = InceptionC()
        self.a9 = nn.AvgPool2d(8)
        self.a10 = nn.Linear(1536, 10)
        
    def forward(self,x):
        out = self.a1(x)
        out = self.a2(out)
        out = self.a3(out)
        out = self.a4(out)
        out = self.a5(out)
        out = self.a6(out)
        out = self.a7(out)
        out = self.a8(out)
        out = self.a9(out)
        out = out.view(out.size(0), -1)
        out = self.a10(out)
        return out
    
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")


net = Inceptionv4()
net = net.to(device)
param = list(net.parameters())
print(len(param))
for i in param:
    print(i.shape)        

98
torch.Size([32, 3, 3, 3])
torch.Size([32])
torch.Size([32, 32, 3, 3])
torch.Size([32])
torch.Size([64, 32, 3, 3])
torch.Size([64])
torch.Size([96, 64, 3, 3])
torch.Size([96])
torch.Size([64, 160, 1, 1])
torch.Size([64])
torch.Size([64, 64, 7, 1])
torch.Size([64])
torch.Size([64, 64, 1, 7])
torch.Size([64])
torch.Size([96, 64, 3, 3])
torch.Size([96])
torch.Size([64, 160, 1, 1])
torch.Size([64])
torch.Size([96, 64, 3, 3])
torch.Size([96])
torch.Size([192, 192, 3, 3])
torch.Size([192])
torch.Size([64, 384, 1, 1])
torch.Size([64])
torch.Size([96, 64, 3, 3])
torch.Size([96])
torch.Size([96, 96, 3, 3])
torch.Size([96])
torch.Size([64, 384, 1, 1])
torch.Size([64])
torch.Size([96, 64, 3, 3])
torch.Size([96])
torch.Size([96, 384, 1, 1])
torch.Size([96])
torch.Size([96, 384, 1, 1])
torch.Size([96])
torch.Size([192, 384, 1, 1])
torch.Size([192])
torch.Size([224, 192, 3, 3])
torch.Size([224])
torch.Size([256, 224, 3, 3])
torch.Size([256])
torch.Size([384, 384, 3, 3])
torch.Size([384])
torch.Siz

데이터셋 로드

In [11]:
transform = transforms.Compose(
    [transforms.Resize(299),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

In [12]:
trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True,transform=transform)

Files already downloaded and verified


In [13]:
trainloader = torch.utils.data.DataLoader(trainset, batch_size=1, shuffle=True, num_workers=2)

In [14]:
testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)

Files already downloaded and verified


In [15]:
testloader = torch.utils.data.DataLoader(testset,batch_size=64, shuffle=False, num_workers=2)

In [16]:
classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

손실함수 정의

In [17]:
import torch.optim as optim

criterion = nn.CrossEntropyLoss().cuda()
optimizer = optim.Adam(net.parameters(), lr=0.0001)

학습하기

In [53]:
for epoch in range(5):
    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        inputs, labels = data
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()
        
        
        outputs = net(inputs)
        break
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
        if i % 100 == 99:
            print('[%d, %5d] loss: %.3f' % (epoch +1, i + 1, running_loss / 100))
            running_loss = 0.0
            
print('Finished Traing')

Finished Traing
